In [25]:
import pandas as pd
import numpy as np
import json
import requests

In [26]:
try:
    response = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
    response.raise_for_status()
    data = response.json()
except requests.exceptions.RequestException as e:
    print(f"There was an error: {e} while retrieving data")
        
df_elements = pd.DataFrame(data["elements"])
#df_element_types = pd.DataFrame(data["element_types"])
#df_element_types
#df_element_types.columns
#df_elements.head(5)
#df_elements[['dreamteam_count', 'in_dreamteam']].head()
    
# top_level = ['events', 'game_settings', 'phases', 'teams', 'elements', 'element_stats', 'element_types']    
df_elements.head()

#events = pd.DataFrame(data["events"])
#events.head(5)
    

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,0.0,0.0,438098,0,0,-1,1,0,3,0.0,...,138,82,676,304,682,310,633,278,0.00,0.00
1,100.0,100.0,205651,0,0,-2,2,0,4,1.6,...,32,15,316,37,462,50,214,36,0.79,0.40
2,75.0,75.0,226597,0,0,2,-2,2,2,3.3,...,66,3,63,18,29,4,13,5,1.08,0.33
3,100.0,100.0,219847,0,0,-1,1,1,4,2.8,...,13,5,178,26,42,14,39,9,1.00,0.31
4,0.0,0.0,463748,0,0,0,0,0,1,0.0,...,640,64,449,50,514,58,556,68,0.00,0.00


In [30]:
import requests
import pandas as pd

#https://fantasy.premierleague.com/a/api/element-summary/438098.jpg
def load_player_data_from_api():
    """Fetches player data from the FPL API and returns a DataFrame with selected columns."""
    try:
        response = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        st.error(f"There was an error: {e} while retrieving data")
        return pd.DataFrame()  # Return empty DataFrame on error

    # Load data into DataFrames
    df_elements = pd.DataFrame(data["elements"])
    df_element_types = pd.DataFrame(data["element_types"])

    # Merge df_elements and df_element_types on 'element_type' and 'id'
    df_merged = df_elements.merge(
        df_element_types[['id', 'plural_name_short']],
        left_on='element_type',
        right_on='id',
        how='left'
    )

    # Rename 'plural_name_short' to 'position'
    df_merged.rename(columns={'plural_name_short': 'position'}, inplace=True)

    # Drop redundant columns
    df_merged.drop(columns=['id_y'], inplace=True)
    df_merged.rename(columns={'id_x': 'id'}, inplace=True)

    # Standardize 'position' to upper case
    df_merged['position'] = df_merged['position'].str.upper()

    # Add 'player_value_score' (custom metric, here set equal to 'total_points')
    df_merged['player_value_score'] = df_merged['total_points']

    # Select the desired columns
    columns_to_use = [
        'id', 'web_name', 'first_name', 'second_name', 'photo', 'code', 'position', 'now_cost',
        'total_points', 'player_value_score', 'minutes', 'goals_scored',
        'assists', 'clean_sheets', 'saves', 'goals_conceded', 'yellow_cards',
        'red_cards', 'saves', 'bonus', 'influence', 'creativity',
        'threat', 'ict_index', 'selected_by_percent', 'form', 'points_per_game'
    ]

    # Ensure all selected columns exist in the DataFrame
    df_final = df_merged[columns_to_use]

    # No data type conversions are performed

    return df_final

player_data = load_player_data_from_api()
#player_data.head(5)

base_photo_url = "https://resources.premierleague.com/premierleague/photos/players/110x140/p"
#player_data['photo_url'] = base_photo_url + player_data['code'].astype(str) + ".png"
player_data['ict_index']

0       0.0
1      16.9
2      59.7
3      83.4
4       0.0
       ... 
688     0.0
689    17.8
690     6.6
691     0.0
692     0.0
Name: ict_index, Length: 693, dtype: object

In [11]:
df_events = pd.DataFrame(data["events"])
#df_events.head()
df_events.columns

Index(['id', 'name', 'deadline_time', 'release_time', 'average_entry_score',
       'finished', 'data_checked', 'highest_scoring_entry',
       'deadline_time_epoch', 'deadline_time_game_offset', 'highest_score',
       'is_previous', 'is_current', 'is_next', 'cup_leagues_created',
       'h2h_ko_matches_created', 'ranked_count', 'chip_plays', 'most_selected',
       'most_transferred_in', 'top_element', 'top_element_info',
       'transfers_made', 'most_captained', 'most_vice_captained'],
      dtype='object')

In [12]:
df_teams = pd.DataFrame(data["teams"])
#df_teams.head()
df_teams.columns

Index(['code', 'draw', 'form', 'id', 'loss', 'name', 'played', 'points',
       'position', 'short_name', 'strength', 'team_division', 'unavailable',
       'win', 'strength_overall_home', 'strength_overall_away',
       'strength_attack_home', 'strength_attack_away', 'strength_defence_home',
       'strength_defence_away', 'pulse_id'],
      dtype='object')

In [13]:
df_element_types = pd.DataFrame(data["element_types"])
#df_element_types
df_element_types.head(5)

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],72
1,2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],231
2,3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],315
3,4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],75


In [14]:
df = pd.read_csv("players.csv")
df.head()
df.columns
'red_cards', 'points_per_game', 

('red_cards', 'points_per_game')

In [15]:
df["position"].unique()

array(['MID', 'FWD', 'DEF', 'GKP'], dtype=object)

In [16]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

# Set up Streamlit page with custom theme
st.set_page_config(
    page_title="Fantasy Premier League",
    page_icon="⚽",
    layout="wide",
    initial_sidebar_state="expanded"
)



# Constants
CSV_FILE_PATH = "players.csv"
APP_TITLE = "Fantasy Premier League"
BUDGET = 1000

# Define formations with positions and required players
FORMATION_MAP = {
    '3-4-3': {'GKP': 1, 'DEF': 3, 'MID': 4, 'FWD': 3},
    '3-5-2': {'GKP': 1, 'DEF': 3, 'MID': 5, 'FWD': 2},
    '4-4-2': {'GKP': 1, 'DEF': 4, 'MID': 4, 'FWD': 2},
    '4-3-3': {'GKP': 1, 'DEF': 4, 'MID': 3, 'FWD': 3},
}

FIELD_COORDS = {
    '3-4-3': {
        'GKP': [(40, 10)],
        'DEF': [(20, 30), (40, 30), (60, 30)],
        'MID': [(10, 60), (30, 60), (50, 60), (70, 60)],
        'FWD': [(20, 90), (40, 90), (60, 90)],
    },
    '3-5-2': {
        'GKP': [(40, 10)],
        'DEF': [(20, 30), (40, 30), (60, 30)],
        'MID': [(10, 60), (25, 60), (40, 60), (55, 60), (70, 60)],
        'FWD': [(30, 90), (50, 90)],
    },
    '4-4-2': {
        'GKP': [(40, 10)],
        'DEF': [(10, 30), (30, 30), (50, 30), (70, 30)],
        'MID': [(10, 60), (30, 60), (50, 60), (70, 60)],
        'FWD': [(30, 90), (50, 90)],
    },
    '4-3-3': {
        'GKP': [(40, 10)],
        'DEF': [(10, 30), (30, 30), (50, 30), (70, 30)],
        'MID': [(20, 50), (40, 50), (60, 50)],
        'FWD': [(10, 90), (40, 90), (70, 90)],
    },
}

# App Title
st.title(APP_TITLE)

# Load and preprocess data
@st.cache_data
def load_player_data(file_path):
    data = pd.read_csv(file_path)
    data['player_value_score'] = data['total_points']  # Assuming total_points is the value metric
    data['position'] = data['position'].str.upper()  # Standardize position values
    return data

player_data = load_player_data(CSV_FILE_PATH)

# Initialize session state for selected players
if 'selected_players' not in st.session_state:
    st.session_state.selected_players = {}

# Sidebar - Team Selection
st.sidebar.title("Team Selection")
formation = st.sidebar.selectbox("Select Formation", list(FORMATION_MAP.keys()), index=0)
position_counts = FORMATION_MAP[formation]

# Function to adjust selected players
def adjust_selected_players(position_counts):
    for position, count in position_counts.items():
        # Get current selected players for the position
        selected = st.session_state.selected_players.get(position, [])

        # If we need to reduce the number of players
        if len(selected) > count:
            # Keep the top 'count' players based on total points
            selected = sorted(selected, key=lambda x: x['total_points'], reverse=True)[:count]
            st.session_state.selected_players[position] = selected
        # If we need to add more players
        elif len(selected) < count:
            # Get available players not already selected
            selected_names = [p['web_name'] for p in selected]
            available_players = player_data[
                (player_data['position'] == position) &
                (~player_data['web_name'].isin(selected_names))
                ]
            # Select top players based on total points
            needed = count - len(selected)
            top_players = available_players.sort_values(by='total_points', ascending=False).head(needed)
            selected.extend(top_players.to_dict('records'))
            st.session_state.selected_players[position] = selected
        # If the number is the same, do nothing


# Function to select players for a position
def select_players_for_position(position, count, remaining_budget):
    """Handles player selection for a specific position with budget constraints."""
    st.sidebar.markdown(f"### {position} Players")

    # Allow user to set a maximum cost for players in this position
    max_cost = st.sidebar.slider(
        f"Set maximum cost for {position} players",
        min_value=int(player_data['now_cost'].min()),
        max_value=int(player_data['now_cost'].max()),
        value=int(player_data['now_cost'].max()),
        key=f"max_cost_{position}"
    )

    # Filter available players based on max_cost and remaining_budget
    available_players = player_data[
        (player_data['position'] == position) &
        (player_data['now_cost'] <= max_cost) &
        (player_data['now_cost'] <= remaining_budget)
        ]

    selected_players = st.session_state.selected_players.get(position, [])
    selected_names = [p['web_name'] for p in selected_players]
    options = available_players['web_name'].tolist()

    selected_names = st.sidebar.multiselect(
        f"Select {count} {position} player(s)",
        options,
        default=selected_names,
        key=position,
        help=f"Select up to {count} {position} player(s).",
        max_selections=count  # Requires Streamlit 1.22 or newer
    )

    # Update selected players in session_state
    selected_players = available_players[available_players['web_name'].isin(selected_names)].to_dict('records')
    st.session_state.selected_players[position] = selected_players

    return selected_players

# Store previous formation
if 'formation' not in st.session_state:
    st.session_state.formation = formation

# Check if formation has changed
formation_changed = formation != st.session_state.formation

# Update stored formation
st.session_state.formation = formation

# Adjust selected players if formation changed
if formation_changed:
    adjust_selected_players(position_counts)

# Collect players by position
selected_players = []
total_cost = 0
remaining_budget = BUDGET

for position, count in position_counts.items():
    position_players = select_players_for_position(position, count, remaining_budget)
    position_cost = sum(player['now_cost'] for player in position_players)
    total_cost += position_cost
    remaining_budget -= position_cost
    selected_players.extend(position_players)

# Create columns for layout
col1, col2 = st.columns([3, 1])  # Adjusted column width ratios to give more space to the field

# Draw soccer field with Plotly in the left column
with col1:
    def draw_soccer_field(selected_team, formation):
        """Draws a soccer field that fills the entire plot area without extra space."""
        field_color = "#6cba7c"  # Soft Grass Green
        line_color = "#ffffff"   # White lines

        # Field dimensions
        field_width = 80
        field_height = 120

        # Create Plotly figure
        fig = go.Figure()

        # Set layout
        fig.update_layout(
            xaxis=dict(
                range=[0, field_width],
                showgrid=False,
                zeroline=False,
                visible=False,
                fixedrange=True,
                domain=[0, 1],  # Fill the entire width
            ),
            yaxis=dict(
                range=[0, field_height],
                showgrid=False,
                zeroline=False,
                visible=False,
                fixedrange=True,
                domain=[0, 1],  # Fill the entire height
            ),
            width=700,    # Set the figure width
            height=1000,   # Set the figure height (adjust as needed)
            margin=dict(l=0, r=0, t=0, b=0),  # Remove all margins
            plot_bgcolor=field_color,
            paper_bgcolor=field_color,
            showlegend=False,
        )

        # Field boundary
        fig.add_shape(type="rect", x0=0, x1=field_width, y0=0, y1=field_height,
                      line=dict(color=line_color, width=2), layer='below')

        # Center line
        fig.add_shape(type="line", x0=0, y0=field_height / 2, x1=field_width, y1=field_height / 2,
                      line=dict(color=line_color, width=2), layer='below')

        # Penalty areas and other field markings
        # Bottom penalty box
        fig.add_shape(type="rect", x0=18, x1=62, y0=0, y1=18,
                      line=dict(color=line_color, width=2), layer='below')
        # Top penalty box
        fig.add_shape(type="rect", x0=18, x1=62, y0=102, y1=120,
                      line=dict(color=line_color, width=2), layer='below')

        # Goal areas
        # Bottom goal box
        fig.add_shape(type="rect", x0=30, x1=50, y0=0, y1=6,
                      line=dict(color=line_color, width=2), layer='below')
        # Top goal box
        fig.add_shape(type="rect", x0=30, x1=50, y0=114, y1=120,
                      line=dict(color=line_color, width=2), layer='below')

        # Center circle
        fig.add_shape(type="circle", x0=30, x1=50, y0=51, y1=69,
                      xref="x", yref="y",
                      line=dict(color=line_color, width=2), layer='below')
        # Center spot
        fig.add_shape(type="circle", x0=39.9, x1=40.1, y0=59.9, y1=60.1,
                      xref="x", yref="y",
                      fillcolor=line_color, line=dict(color=line_color, width=2), layer='below')

        # Penalty spots
        # Bottom penalty spot
        fig.add_shape(type="circle", x0=39.9, x1=40.1, y0=11.9, y1=12.1,
                      xref="x", yref="y",
                      fillcolor=line_color, line=dict(color=line_color, width=2), layer='below')
        # Top penalty spot
        fig.add_shape(type="circle", x0=39.9, x1=40.1, y0=107.9, y1=108.1,
                      xref="x", yref="y",
                      fillcolor=line_color, line=dict(color=line_color, width=2), layer='below')

        # Corner arcs (optional)

        # Add players to the field
        coords = FIELD_COORDS[formation]
        for position, spots in coords.items():
            players = [p for p in selected_team if p['position'] == position]
            for i, (x, y) in enumerate(spots):
                if i < len(players):
                    player = players[i]
                    fig.add_trace(go.Scatter(
                        x=[x], y=[y],
                        mode="markers+text",
                        marker=dict(size=25, color="#e90052", line=dict(width=2, color="#04f5ff")),
                        text=player['web_name'],
                        textposition="bottom center",
                        textfont=dict(size=11, color="#ffffff", family="Arial"),
                        hovertemplate=(
                            f"<b>{player['web_name']}</b><br>"
                            f"Position: {player['position']}<br>"
                            f"Cost: {player['now_cost']}<br>"
                            f"Value: {player['player_value_score']}<extra></extra>"
                        ),
                        showlegend=False
                    ))
                else:
                    # Placeholder markers
                    fig.add_trace(go.Scatter(
                        x=[x], y=[y],
                        mode="markers",
                        marker=dict(size=20, color="#808080", line=dict(width=2, color="#ffffff")),
                        hoverinfo="skip",
                        showlegend=False
                    ))

        return fig

    field_fig = draw_soccer_field(selected_players, formation)
    st.plotly_chart(field_fig, use_container_width=True)

# Display Selected Team and Budget in the right column
with col2:
    st.subheader("Selected Team")
    if selected_players:
        team_df = pd.DataFrame(selected_players)
        st.table(team_df[['web_name', 'position', 'now_cost', 'player_value_score']].style.hide_index())

        # Budget progress bar
        budget_usage = total_cost / BUDGET
        st.progress(budget_usage, text=f"Budget Usage: {total_cost} / {BUDGET}")

        if total_cost > BUDGET:
            st.error("Budget exceeded!")
    else:
        st.info("No players selected.")
  

2024-12-07 15:05:16.735 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 15:05:16.736 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 15:05:16.763 
  command:

    streamlit run /Users/richardpallangyo/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-12-07 15:05:16.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 15:05:16.764 No runtime found, using MemoryCacheStorageManager
2024-12-07 15:05:16.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 15:05:16.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-